In [1]:
!pip install espn_api
!pip install pygsheets
from espn_api.basketball import League
from datetime import datetime, timedelta
import pandas as pd

  Using cached espn_api-0.44.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
Using cached espn_api-0.44.1-py3-none-any.whl (66 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
  Using cached pygsheets-2.0.6-py3-none-any.whl.metadata (14 kB)
Using cached pygsheets-2.0.6-py3-none-any.whl (158 kB)


In [2]:
import pygsheets
def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True, index = False)
    wks_write.frozen_rows = 1
serv_path = 'keykey.json'
SPREADSHEET_ID = '1Ku6qmPGoKjZ16NUBLCdopkNI-qw4BnseQGDUQXa_oko'
sheet_name = 'My team API'
#write_to_gsheet(serv_path, SPREADSHEET_ID, sheet_name, quinn_m)

In [3]:
league = League(league_id=403382108, year=2025, espn_s2='AECe11iWbFkute%2FcY4wPm5PnP5LUWP13PANIDlEASVbSasrkLRJ15a%2F%2FtRwmnMpFI%2FSSHhyqR1X4GntAc6HshqRDwVFmzrrR52X5Bq8qcXk1ryfkfHcfqkRQA6p8%2Bf4Yw5tpjgS7j2ceNsMjz4GGzYKGo3a2%2FvTXOCszrRK%2BDA5J0QXu9jBhyaoaas83MQ%2F%2BZMQbG6yvVyVbovujmTCxmIvcZf1UPDsRa5zaYubuxQQdznBqXjVjl%2Ff3V6VGdwolqNEbEmsQFItF9ROaX0GSPozOgSsRn5LmeDXSx6fEt%2FfxjA%3D%3D', swid='{FC544995-8240-4FE8-9449-958240FFE88D}')

In [4]:
matchups = league.scoreboard()

# Loop through matchups to find starting players
starters = {}
for matchup in matchups:

    home_starters = []
    away_starters = []
    for player in matchup.home_team.roster:
        if player.lineupSlot not in ['BE', 'IR']:  # Exclude bench (BE) and injured (IR) slots
            home_starters.append(player)
    if matchup.away_team != 0: 
        for player in matchup.away_team.roster:
            if player.lineupSlot not in ['BE', 'IR']:
                away_starters.append(player)

    starters[matchup.home_team] = home_starters
    starters[matchup.away_team] = away_starters


In [5]:
#daily 2
livescores = pd.read_pickle("livescores.pkl")
def scoreupdater():
    daybefore = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)+timedelta(-1)
    daybefore = daybefore.strftime('%Y-%m-%d')
    #print(daybefore in days)
    for i in league.box_scores():
        home = str(i.home_team)
        home_score = i.home_score 
        away = str(i.away_team)
        away_score = i.away_score
        livescores.loc[home, daybefore] = home_score
        livescores.loc[away, daybefore] = away_score
        livescores.to_pickle("livescores.pkl")
    return livescores
scoreupdater()


# Reload the DataFrame

livescores2 = livescores.copy()
livescores2.reset_index(inplace=True)

write_to_gsheet(serv_path, SPREADSHEET_ID, 'Livescores', livescores2)

/srv/conda/lib/python3.11/site-packages/pygsheets/worksheet.py:1431: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(nan)


In [6]:
#daily one
def final_prod(team):

    #today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    currentmon = (datetime.today() - timedelta(days=datetime.today().weekday())).replace(hour=0, minute=0, second=0, microsecond=0)
    today = currentmon
    nextmon = today + timedelta(days=7)
    days = []
    for i in range(7):
        days.append((today + timedelta(days=i)))
    def date_extracter(roster):
        date_dict = {}
        for i in roster:
            sched = list(dict(sorted(i.schedule.items(), key=lambda item: int(item[0]))).values())
            games = []
            for game in sched:
                if nextmon>game['date']> today:
                    games.append(game['date'].replace(hour=0, minute=0, second=0, microsecond=0))
            date_dict[i.name] = games
    
        return date_dict

    def matrix_maker(games, day_list):
    
        # Initialize an empty list to store the matrix
        matrix = []
        
        # Iterate through each person and create their row in the matrix
        for person, dates in games.items():
            row = []
            for date in day_list:
                # Append 1 if the person is going on that date, otherwise append 0
                if date in dates:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        #check starters
        # Convert the matrix to a DataFrame for better visualization
        game_matrix = pd.DataFrame(matrix, columns=day_list, index=games.keys())
        game_matrix['Average Points'] = [i.avg_points for i in starters[team]]
        others = list(game_matrix.columns[0:14])
        others.insert(0,'Average Points')
        return game_matrix[others]

    games = date_extracter(starters[team])
    fin = matrix_maker(games,days)
    rn = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    for i in starters[team]:
        if i.injured == True:
            fin.loc[i.name, rn] = 0
    fin.columns = fin.columns.astype(str)
    fin.reset_index(inplace=True)
    fin = fin.rename(columns = {'index': team})
    fin.columns = fin.columns.astype(str)
    

    return fin

#to sheets
for i in league.teams:
    write_to_gsheet(serv_path, SPREADSHEET_ID, str(i), final_prod(i))

